##### Imports and Data

In [60]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
from sklearn import model_selection
import seaborn as sns

from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import dummy
from sklearn import tree
from sklearn import neighbors
from sklearn import discriminant_analysis
from sklearn import svm
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

In [61]:
RANDOM_SEED = 42069360
db = pd.read_csv('fifa_final.csv')
print(db.describe())
# Feature Set for Predicting Card Level / Color
card_level_db = db[db.min_played >= 90] 
card_level_db = pd.concat([card_level_db.iloc[:, 7:40].drop(columns=['ws_season']), card_level_db.iloc[:, -1]], axis=1)

# Card Labels for Level / Color
card_labels = card_level_db.fifa_overall_category.map({'bronze': 0, 'silver': 1, 'gold': 2})

# Card Features for Level / Color
card_features = card_level_db.drop(columns=['fifa_overall_category'])
card_features = pd.get_dummies(card_features, drop_first=True, columns=['ws_tournament_name', 'position'])

# Splitting training and testing data, hard 80/20 Split
train_card_features, test_card_features, train_card_labels, test_card_labels = model_selection.train_test_split(card_features, card_labels, test_size=0.2, random_state=RANDOM_SEED)
# Card Groups for Level / Color, Training
train_card_groups = train_card_features.ws_player_id
train_card_features = train_card_features.drop(columns=['ws_player_id'])

# Card Groups for Level / Color, Testing
test_card_groups = test_card_features.ws_player_id
test_card_features = test_card_features.drop(columns=['ws_player_id'])

train_data = train_card_features.copy()
train_card_features.describe()

        Unnamed: 0           id  ...  fifa_aggression  fifa_overall_score
count  20873.00000  20873.00000  ...     20873.000000         20873.00000
mean   10436.00000  10437.00000  ...        62.393092            71.80252
std     6025.66042   6025.66042  ...        16.665678             6.28183
min        0.00000      1.00000  ...        11.000000            47.00000
25%     5218.00000   5219.00000  ...        53.000000            68.00000
50%    10436.00000  10437.00000  ...        66.000000            72.00000
75%    15654.00000  15655.00000  ...        75.000000            76.00000
max    20872.00000  20873.00000  ...        96.000000            94.00000

[8 rows x 72 columns]


,age,height,weight,appearances,subs_on,min_played,goal_per_game,assists_total_per_game,yellow_cards_per_game,red_cards_per_game,shots_per_game,aerials_won_per_game,man_of_match_per_game,pass_success,tackles_per_game,interceptions_per_game,fouls_per_game,offsides_won_per_game,clearance_per_game,was_dribbled_per_game,outfielder_blocked_per_game,goal_own_per_game,key_pass_per_game,dribbles_won_per_game,fouls_given_per_game,offsides_given_per_game,dispossessed_per_game,turnovers_per_game,total_passes_per_game,ws_tournament_name_Bundesliga,ws_tournament_name_Bundesliga II,ws_tournament_name_Champions League,ws_tournament_name_Eredivisie,ws_tournament_name_Europa League,ws_tournament_name_LaLiga,ws_tournament_name_League One,ws_tournament_name_League Two,ws_tournament_name_Liga NOS,ws_tournament_name_Ligue 1,ws_tournament_name_Major League Soccer,ws_tournament_name_Premier League,ws_tournament_name_Serie A,ws_tournament_name_Super Lig,ws_tournament_name_Super league,ws_tournament_name_Superliga,ws_tournament_name_UEFA Nations League A,position_CB,position_CDM,position_CF,position_CM,position_GK,position_LB,position_LF,position_LM,position_LW,position_LWB,position_RB,position_RF,position_RM,position_RW,position_RWB,position_ST
count,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000,15549.000000
mean,28.304650,181.809827,76.305936,20.830278,3.948228,1512.860441,0.084559,0.060634,0.140497,0.008134,0.818170,1.179992,0.031467,76.230561,1.210830,0.900366,0.933831,0.147143,1.503382,0.621746,0.212004,0.000064,0.611780,0.605459,0.850988,0.128276,0.701410,1.001887,30.021176,0.084121,0.051386,0.010997,0.064377,0.042575,0.096855,0.002894,0.009968,0.063412,0.086951,0.086629,0.100650,0.091774,0.070873,0.044955,0.080455,0.001093,0.184642,0.088945,0.007010,0.115892,0.084314,0.079298,0.000064,0.057882,0.023731,0.003730,0.081356,0.000064,0.059875,0.020194,0.003537,0.134221
std,4.326582,6.650296,7.016050,11.133984,4.476860,981.922290,0.131003,0.083150,0.123170,0.026422,0.747191,1.082731,0.055504,9.865642,0.842376,0.768591,0.574338,0.259036,1.574990,0.468875,0.271796,0.008020,0.550288,0.612937,0.621470,0.221581,0.616962,0.773844,14.317383,0.277578,0.220791,0.104294,0.245431,0.201904,0.295770,0.053720,0.099347,0.243711,0.281772,0.281300,0.300874,0.288716,0.256620,0.207211,0.272006,0.033048,0.388019,0.284673,0.083435,0.320105,0.277867,0.270212,0.008020,0.233527,0.152216,0.060963,0.273389,0.008020,0.237263,0.140669,0.059371,0.340900
min,17.000000,156.000000,55.000000,1.000000,0.000000,90.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.814815,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,177.000000,71.000000,11.000000,0.000000,643.000000,0.000000,0.000000,0.047619,0.000000,0.277778,0.400000,0.000000,71.428571,0.583333,0.250000,0.548246,0.000000,0.363636,0.280000,0.000000,0.000000,0.196429,0.166667,0.400000,0.00

## Models

In [64]:


clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 3), max_iter=10000)

clf.fit(train_card_features, train_card_labels)



MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 3), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [65]:
regr = MLPRegressor(max_iter=500).fit(train_card_features, train_card_labels)


## Results

In [67]:
target_names = ['bronze', 'silver', 'gold']

preds = clf.predict(test_card_features)
probs = clf.predict_proba(test_card_features)
regprobs = clf.predict(test_card_features)

clf.score(test_card_features, test_card_labels)
print("NN Classifier Model Report (Avg):")
print(metrics.classification_report(test_card_labels, preds, target_names=target_names))
print(f"Balanced Accuracy: {metrics.balanced_accuracy_score(test_card_labels, preds)}")
print(f"ROC AUC Values: {metrics.roc_auc_score(test_card_labels, probs, multi_class='ovr', average='weighted')}")
print("-----------------------------------------")

print("NN Regressor Model Report (Avg):")
metrics.r2_score(test_card_labels, regprobs)


NN Classifier Model Report (Avg):
              precision    recall  f1-score   support

      bronze       0.00      0.00      0.00       362
      silver       0.55      1.00      0.71      2129
        gold       0.00      0.00      0.00      1397

    accuracy                           0.55      3888
   macro avg       0.18      0.33      0.24      3888
weighted avg       0.30      0.55      0.39      3888

Balanced Accuracy: 0.3333333333333333
ROC AUC Values: 0.5
-----------------------------------------
NN Regressor Model Report (Avg):


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-0.1857261224317832